In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (145, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


In [4]:
df_annotation.sample(10)

,width,height,slice,vcoordX,vcoordY,vcoordZ,diameterX,diameterY,diameterZ,originX,originY,originZ,spacingX,spacingY,spacingZ,label
seriesuid,,,,,,,,,,,,,,,,
696405,347,347,324,222.296875,238.562500,287.0,7.455078,11.521484,9.976188,-191.161133,-313.161133,-717.838013,0.677734,0.677734,4.988094,5
656852,361,361,335,225.977539,236.906250,187.0,8.460938,9.166016,10.000000,-157.147461,-325.147461,-706.799988,0.705078,0.705078,5.000000,32
414316,339,339,320,245.308056,110.239653,110.0,42.374401,51.643801,15.000000,-169.169006,700.611023,729.890015,0.662100,0.662100,5.000000,5
340229,327,327,350,92.288086,64.825195,115.0,5.109375,8.941406,5.000000,-198.000000,-163.500000,-737.700012,0.638672,0.638672,5.000000,5
689863,350,350,255,246.093750,217.382812,65.0,14.355469,17.089844,5.000000,-171.753006,-174.658203,-334.039307,0.683594,0.683594,5.000000,5
525840,380,380,230,207.070312,204.843750,125.0,12.617188,8.164062,20.000000,-187.000000,-190.000000,-182.000000,0.742188,0.742188,10.000000,31
369542,350,350,306,199.951172,221.484375,219.0,8.203125,6.152344,1.000000,-158.899994,-50.400002,-394.850006,0.683594,0.683594,1.000000,31
543544,402,402,315,105.996094,199.037109,235.0,10.207031,7.851562,5.000000,-196.249664,-200.607422,-406.118896,0.785156,0.785156,5.000000,1
639087,350,350,285,91.601563,225.927734,145.0,6.152344,4.101562,5.000000,-172.479324,-174.658203,1234.204834,0.683594,0.683594,5.000000,1


# 2. Load Center Coords by Candidates

In [5]:
lungs = list(set(df_meta_test.index))
print('distinct lungs in testset:', len(lungs))

distinct lungs in testset: 145


In [6]:
list_tasks = []
size = settings.CUBE_SIZE
step = 32
start = [ size//2, size//2, size//2 ] # z,y,x

for uid in tqdm(lungs):
    meta = df_meta_test.loc[uid] # slice
    
    for z in range(start[0], int(meta.slice), step):
        for y in range(start[1], int(meta.height), step):
            for x in range(start[2], int(meta.width), step):
                task = {}
                task['seriesuid'] = uid
                task['vcoordX'] = x
                task['vcoordY'] = y
                task['vcoordZ'] = z
                
                # print(task)
                list_tasks.append(task)

100%|██████████| 145/145 [00:00<00:00, 314.95it/s]


In [7]:
df_tasks = pd.DataFrame(list_tasks, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
df_tasks = df_tasks.set_index('seriesuid')
df_tasks.index = df_tasks.index.astype('str')

print('total:', df_tasks.shape, 'lung:', len(set(df_tasks.index)))

if not os.path.exists(settings.SUBMISSION_DIR + 'candidates'):
    os.mkdir(settings.SUBMISSION_DIR + 'candidates')
    
df_tasks.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_cubes.csv', encoding='utf-8')


total: (142434, 3) lung: 145


# 3. Predict Results by tasks_cubes.csv

In [8]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

label = '5'
output_dir = './output/unet3d/1564290301/' # 1. 1564290030 5. 1564290301
model = load_model(output_dir + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

W0728 23:54:17.216179 139911712622400 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0728 23:54:17.218051 139911712622400 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0728 23:54:17.224705 139911712622400 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [10]:
batch_size = 32
threshold_probability = 0.8
wtype = 'lung'
plot = False

In [11]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [12]:
list_results = []
print(wtype + 'window')
for uid in set(df_tasks.index):
    tasks = df_tasks.loc[[uid]] # DataFrame
    meta = df_meta_test.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks), 'meta:', meta.slice, meta.height, meta.width)
#     print(meta)
                 

    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='test', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=settings.CUBE_SIZE)

        if np.sum(cube) > 0:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            predictions_test = model.predict(X_test, batch_size=batch_size, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                pred = predictions_test[i] > threshold_probability
                if np.sum(pred) > 10:
                    vcoords, diameters, bboxes = helper.get_regions_detail(pred[:,:,:,0], itm)
                    if len(bboxes) > 0:
                        for i, box in enumerate(bboxes):
                            result = {}
                            result['seriesuid'] = uid
                            result['vcoordX'] = vcoords[i][2]
                            result['vcoordY'] = vcoords[i][1]
                            result['vcoordZ'] = vcoords[i][0]
                            result['diametersX'] = diameters[i][2]
                            result['diametersY'] = diameters[i][1]
                            result['diametersZ'] = diameters[i][0]
                            if plot:
                                plot_cube((pred*255).astype(np.uint8))

                            list_results.append(result)
                        
        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','vcoordX','vcoordY','vcoordZ','diametersX','diametersY','diametersZ'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)))

    df_results.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype + '_' + label +'.csv', encoding='utf-8')

    print('-'*100)
    

lungwindow
lung: 632110 tasks: 800 meta: 270.0 350.0 350.0
511/511 [==============================] - 15s 29ms/sample
total: (58, 6) lung: 1
----------------------------------------------------------------------------------------------------
lung: 697306 tasks: 800 meta: 260.0 322.0 322.0
515/515 [==============================] - 8s 16ms/sample
total: (115, 6) lung: 2
----------------------------------------------------------------------------------------------------
lung: 627957 tasks: 900 meta: 315.0 350.0 350.0
524/524 [==============================] - 10s 20ms/sample
total: (162, 6) lung: 3
----------------------------------------------------------------------------------------------------
lung: 655508 tasks: 700 meta: 230.0 322.0 322.0
419/419 [==============================] - 9s 21ms/sample
total: (199, 6) lung: 4
----------------------------------------------------------------------------------------------------
lung: 371893 tasks: 729 meta: 310.0 315.0 315.0
591/591 [=======

559/559 [==============================] - 10s 17ms/sample
total: (2023, 6) lung: 36
----------------------------------------------------------------------------------------------------
lung: 696355 tasks: 900 meta: 315.0 350.0 350.0
619/619 [==============================] - 13s 21ms/sample
total: (2081, 6) lung: 37
----------------------------------------------------------------------------------------------------
lung: 664055 tasks: 1440 meta: 350.0 386.0 386.0
749/749 [==============================] - 16s 21ms/sample
total: (2148, 6) lung: 38
----------------------------------------------------------------------------------------------------
lung: 663949 tasks: 600 meta: 216.0 330.0 330.0
368/368 [==============================] - 6s 16ms/sample
total: (2195, 6) lung: 39
----------------------------------------------------------------------------------------------------
lung: 505792 tasks: 1008 meta: 235.0 416.0 416.0
636/636 [==============================] - 12s 19ms/sample
tota

723/723 [==============================] - 10s 14ms/sample
total: (3970, 6) lung: 71
----------------------------------------------------------------------------------------------------
lung: 506836 tasks: 600 meta: 216.0 332.0 332.0
394/394 [==============================] - 5s 14ms/sample
total: (4039, 6) lung: 72
----------------------------------------------------------------------------------------------------
lung: 522276 tasks: 1296 meta: 290.0 402.0 402.0
632/632 [==============================] - 9s 14ms/sample
total: (4109, 6) lung: 73
----------------------------------------------------------------------------------------------------
lung: 394788 tasks: 512 meta: 280.0 285.0 285.0
433/433 [==============================] - 6s 14ms/sample
total: (4168, 6) lung: 74
----------------------------------------------------------------------------------------------------
lung: 400230 tasks: 1152 meta: 280.0 394.0 394.0
651/651 [==============================] - 9s 14ms/sample
total: 

522/522 [==============================] - 7s 13ms/sample
total: (6027, 6) lung: 106
----------------------------------------------------------------------------------------------------
lung: 530088 tasks: 1210 meta: 350.0 380.0 380.0
588/588 [==============================] - 8s 13ms/sample
total: (6128, 6) lung: 107
----------------------------------------------------------------------------------------------------
lung: 514106 tasks: 640 meta: 336.0 274.0 274.0
497/497 [==============================] - 6s 13ms/sample
total: (6221, 6) lung: 108
----------------------------------------------------------------------------------------------------
lung: 689003 tasks: 800 meta: 265.0 350.0 350.0
429/429 [==============================] - 6s 14ms/sample
total: (6252, 6) lung: 109
----------------------------------------------------------------------------------------------------
lung: 322782 tasks: 1089 meta: 292.0 360.0 360.0
463/463 [==============================] - 7s 14ms/sample
tota

412/412 [==============================] - 6s 14ms/sample
total: (8208, 6) lung: 141
----------------------------------------------------------------------------------------------------
lung: 672232 tasks: 900 meta: 320.0 340.0 340.0
620/620 [==============================] - 9s 15ms/sample
total: (8248, 6) lung: 142
----------------------------------------------------------------------------------------------------
lung: 504771 tasks: 1452 meta: 408.0 383.0 383.0
748/748 [==============================] - 11s 15ms/sample
total: (8321, 6) lung: 143
----------------------------------------------------------------------------------------------------
lung: 688247 tasks: 1210 meta: 340.0 371.0 371.0
767/767 [==============================] - 11s 14ms/sample
total: (8395, 6) lung: 144
----------------------------------------------------------------------------------------------------
lung: 676614 tasks: 900 meta: 290.0 350.0 350.0
606/606 [==============================] - 9s 15ms/sample
to

In [15]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype + '_' + label +'.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))



results: (8447, 6) distinct lung: 145


In [17]:
filepath_1 = settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'_1.csv'
filepath_5 = settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'_5.csv'
if os.path.exists(filepath_1) and os.path.exists(filepath_5):
    df_lung_1 = pd.read_csv(filepath_or_buffer=filepath_1, index_col=['seriesuid'])
    df_lung_1.index = df_lung_1.index.astype('str')
    print('lung window 1:', df_lung_1.shape)
    
    df_lung_5 = pd.read_csv(filepath_or_buffer=filepath_5, index_col=['seriesuid'])
    df_lung_5.index = df_lung_5.index.astype('str')
    print('lung window 5:', df_lung_5.shape)
    
    df_lung = pd.concat([df_lung_1, df_lung_5])
    print('lung windows merged:', df_lung.shape)
    df_lung.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'.csv', encoding='utf-8')

lung window 1: (3503, 6)
lung window 5: (8447, 6)
lung windows merged: (11950, 6)
